# Practicing SQL by Answering Real Life Business Problems

## Introduction

In this notebook we will work with the Chinook database. This is a collection of data which is supposed to represent a mock digital media store, much like iTunes. The database is hosted by [this GitHub repository](https://github.com/lerocha/chinook-database) and includes information about artists, albums, media tracks, customers and purchases made on the store. 

We will play the role of a consultant who has been asked to address some business questions related to the store's activity; retrieving the relevant data from the database using SQL, we answer these questions by formulating a strategy which could maximize the revenues. 

The database organizes the data using eleven different tables. We include below a copy of the database schema. For writing our queries, we will tacitly refer to this schema throughout the notebook.

![db_schema](chinook_schema.png)

In order to be able to connect to the database and run SQL code directly on this notebook, we first need to install the appropriate component.  

In [1]:
# Install the %sql magic which allows to run SQL syntax from this notebook
!pip install ipython-sql

In [2]:
# Import the actual SQL module 
%load_ext sql

From now on, in order to execute an SQL query, it is enough to include the magic command `%%sql` before the text of the query (notice that we use two *%* signs so that the entire content of a cell, rather than a single line, will be interpreted as SQL syntax).

---

## Exploring the Database

Before writing our first query, we need to connect to the database.

In [3]:
# Connect to database
%sql sqlite:///chinook.db

Let's start by printing a list of all the tables in the database.

In [4]:
%%sql

-- Print the names of all tables and all views in the database

SELECT
       name,
       type
  FROM sqlite_master
 WHERE type IN ("table","view");

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


Notice how the output is displayed as a pandas dataframe: this is one of the cool features of the *sql* module for Jupyter notebook!

Now, let's write a few basic queries to get familiar with the data.

In [5]:
%%sql

-- Print the first 5 rows of the track table

SELECT *
  FROM track
 LIMIT 5;

 * sqlite:///chinook.db
Done.


track_id,name,album_id,media_type_id,genre_id,composer,milliseconds,bytes,unit_price
1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Hoffman",230619,3990994,0.99
4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. Dirkscneider & W. Hoffman",252051,4331779,0.99
5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99


In [6]:
%%sql

-- Print the number of tracks contained in the track table

SELECT COUNT(*) AS number_of_tracks
  FROM track;

 * sqlite:///chinook.db
Done.


number_of_tracks
3503


In [7]:
%%sql

-- Print the name of the first 10 playlists together with the number of tracks they contain

SELECT p.name Playlist,
       COUNT(t.name) Numbers_of_Track
  FROM playlist p 
       INNER JOIN playlist_track pt ON p.playlist_id = pt.playlist_id
       INNER JOIN track t ON pt.playlist_id=t.track_id
GROUP BY 1
 LIMIT 10;

 * sqlite:///chinook.db
Done.


Playlist,Numbers_of_Track
90’s Music,1477
Brazilian Music,39
Classical,75
Classical 101 - Deep Cuts,25
Classical 101 - Next Steps,25
Classical 101 - The Basics,25
Grunge,15
Heavy Metal Classic,26
Music,6580
Music Videos,1


## Choosing What New Artist to Promote

Let's assume the following scenario: the Chinook record store has signed a deal with a record label to promote some new artist; the record label has provided a short list of four artists but, following the deal, only three albums will be added to the store. Here is some info about these artists:

|**Artist Name**|**Genre**|
|---------------|---------|
|Regal          |Hip-Hop  |
|Red Tone       |Punk     |
|Meteor and the Girls|Pop |
|Slim Jim Bites |Blues    |

We will base our decision on what artists to choose by considering the most popular genres on the platform. Moreover, since the record label is only interested in the American market, we will only considered purchases made in the US.

In [8]:
%%sql

-- Print the number of purchases made in the US for each music genre

SELECT g.name Genre,
       COUNT(il.invoice_line_id) Purchases
FROM genre g
     INNER JOIN track t ON g.genre_id=t.genre_id
     INNER JOIN invoice_line il ON t.track_id=il.track_id
     INNER JOIN invoice i ON il.invoice_id=i.invoice_id
     INNER JOIN customer c ON c.customer_id=i.customer_id
WHERE c.country="USA" AND i.billing_country="USA"
GROUP BY 1
ORDER BY 2 DESC;

 * sqlite:///chinook.db
Done.


Genre,Purchases
Rock,561
Alternative & Punk,130
Metal,124
R&B/Soul,53
Blues,36
Alternative,35
Pop,22
Latin,22
Hip Hop/Rap,20
Jazz,14


Based on these results, we should exclude the artist named Regal. In fact, he sings hip-hop, a genre that is 
not very popular in the Chinook store when compared to the genres of the other artists proposed by the record label.

## Finding the Best Sales Support Agent

In the following, we will compare the different sales support agent by considering how much money their customers have spent.

In [9]:
%%sql

-- Print the name and last name of the employees whose customers spent the most

SELECT e.first_name || " " || e.last_name Sales_agent,
       ROUND(SUM(i.total)) Sales,
       COUNT(DISTINCT(c.customer_id)) Number_of_customers,
       e.hire_date
FROM employee e
     INNER JOIN customer c ON e.employee_id=c.support_rep_id
     INNER JOIN invoice i ON c.customer_id=i.customer_id
GROUP BY 1
ORDER BY 2 DESC;

 * sqlite:///chinook.db
Done.


Sales_agent,Sales,Number_of_customers,hire_date
Jane Peacock,1732.0,21,2017-04-01 00:00:00
Margaret Park,1584.0,20,2017-05-03 00:00:00
Steve Johnson,1394.0,18,2017-10-17 00:00:00


From the table above, it semms like Jane Peacock is the best sale agent since her customers spent the largest amount of money. However, it is important to notice that the lower sales recorded by the other two agents are justified by the fact that they have been working for the Chinook store for a shorter time. All in all, taking in consideration the level of seniority, it's not wrong to say that all the agents are performing equally well.

## Analyzing Sales Data per Country

In this section we will retrieve summary information about sales on the Chinook store in different countries around the world.

In [10]:
%%sql

-- Get various sales data in all different countries where the Chinook store is available

WITH country_info AS(
    SELECT c.country,
           ROUND(SUM(i.total)) total_sales,
           COUNT(DISTINCT(c.customer_id)) total_customers,
           COUNT(i.invoice_id) total_orders,
           ROUND(ROUND(SUM(i.total))/COUNT(i.invoice_id)) avg_sale,
           ROUND(SUM(i.total))/COUNT(DISTINCT(c.customer_id))
           customer_lifetime_value
      FROM customer c
           INNER JOIN invoice i ON c.customer_id=i.customer_id
    GROUP BY 1
    ORDER BY 2 DESC)

-- Print this sales data grouping by country

SELECT *
  FROM
        (SELECT *   -- On one hand, keep countries with at least two customers separate
           FROM country_info
          WHERE total_customers > 1

         UNION

         SELECT "Other", -- On the other hand, group together all the countries with only a single customer  
                 SUM(total_sales),
                 SUM(total_customers),
                 SUM(total_orders),
                 ROUND(SUM(total_sales)/SUM(total_orders)),
                 ROUND(SUM(total_sales)/SUM(total_customers))
           FROM  country_info
          WHERE  total_customers = 1
        )
ORDER BY -- Let the `Other` countries appear on the bottom and order the rest of them by total sales revenue 
         CASE
              WHEN country = "Other" THEN 1  /*Picks out the `Other` nations using a numeric label in order to
                                               pushing them to the bottom of the table*/ 
              ELSE 2
         END DESC,
         total_sales DESC;

 * sqlite:///chinook.db
Done.


country,total_sales,total_customers,total_orders,avg_sale,customer_lifetime_value
USA,1040.0,13,131,8.0,80.0
Canada,536.0,8,76,7.0,67.0
Brazil,428.0,5,61,7.0,85.6
France,389.0,5,50,8.0,77.8
Germany,335.0,4,41,8.0,83.75
Czech Republic,273.0,2,30,9.0,136.5
United Kingdom,246.0,3,28,9.0,82.0
Portugal,185.0,2,29,6.0,92.5
India,183.0,2,21,9.0,91.5
Other,1093.0,15,147,7.0,73.0


USA, Canada and Brazil are the three countries from which the store has racked up the majority of its sales revenues. The highest average sales, however, come from Czech Republic, United Kingdom and India. This suggests that there might be room for growth in these markets and we should therefore invest there.

This said, we need to be a little careful: the sales data in the latter threee countries is generated by a very small customer population (either 2 or 3 customers). If one or more of these customer, for whatever reason, decide to stop using the store, then the sales number could plummet drastically and our investments might cause a loss of money to the store.

## Analyzing Full Album vs. Single Track Sale Strategy

Let's once again assume a hypothetical scenario: in order to save money, management is considering changing the way tracks are purchased from record companies. Under the proposed strategy, only the most popular tracks from each album, rather than the album in its entirety, will be purchased.

The possible downside of this strategy is that customers won't be able to find complete albums on the Chinook store anymore. This might eventually result in a loss of customer and, consequently, loss of revenue. 

Our goal is to determine which of the two strategies (the current one of purchasing all tracks from an album, or the proposed one of only purchasing the most popular ones) is more viable from a business perspective.

To get an answer, we calculate the percentage of customers who purchase entire albums, rather than mixing and matching tracks from different ones. To make things a little easier, we operate under the assumption that, once they have selected an entire album to purchase, customers cannot add extra songs from different albums to their order. 

In [11]:
%%sql

-- We create a temporary table which establishes whether each invoice was an album purchase or not

WITH purchase_info AS(
       SELECT i.invoice_id, -- Get the invoices' ids
    
       /*Now we label this invoice comparing the set of all tracks purchased and the set of all tracks in one
        of the albums one of these tracks appears. Given our assumptions, it doesn't matter which song (hence
        which album) we pick.*/
       CASE WHEN
            (
            SELECT t.track_id   -- Get all tracks contained in a album...    
              FROM track t
             WHERE t.album_id = 
                                (SELECT t.album_id  -- ... from which at least one song was purchased 
                                   FROM invoice_line il 
                                        INNER JOIN track t ON il.track_id=t.track_id
                                  WHERE il.invoice_id = i.invoice_id
                                  LIMIT 1)   
            EXCEPT  -- Now remove... 
            
            SELECT t.track_id  -- ...all tracks purchased in the current invoice    
              FROM track t
                   INNER JOIN invoice_line il ON t.track_id = il.track_id
             WHERE il.invoice_id = i.invoice_id
            ) IS NULL
    
            AND
            
            -- Now perform the same set complement operation of above but inverting the roles of the two sets
            (
            SELECT t.track_id
              FROM track t
                   INNER JOIN invoice_line il ON t.track_id = il.track_id
             WHERE il.invoice_id = i.invoice_id
                
            EXCEPT
                
            SELECT t.track_id
              FROM track t
             WHERE t.album_id = 
                                (SELECT t.album_id
                                   FROM invoice_line il 
                                        INNER JOIN track t ON il.track_id=t.track_id
                                  WHERE il.invoice_id = i.invoice_id
                                  LIMIT 1
                                )
            ) IS NULL    
            
              THEN "Album purchase"   /*If both operations above yield the empty set. it means the two sets 
                                        are the same, i.e., the invoice contains all the songs from an album
                                        hence the purchase is an album purchase...*/
              ELSE "Not album purchase"  -- ... otherwise is not an album purchase
            END type_of_purchase
         FROM invoice i
                                )

-- Now use the temporary table we created to calculate the frequency each album/not album purchases  

SELECT type_of_purchase,
       COUNT(*) number_of_purchases,
       CAST(COUNT(*) AS FLOAT) / (  
                                  SELECT COUNT(*)
                                    FROM invoice) * 100 percentage
  FROM purchase_info
  GROUP BY type_of_purchase 
  ORDER BY number_of_purchases DESC;

 * sqlite:///chinook.db
Done.


type_of_purchase,number_of_purchases,percentage
Not album purchase,500,81.43322475570032
Album purchase,114,18.566775244299674


Only in 18.6% of the cases customers purchased a full album. If we decide to only make available the most popular tracks from each album, then we might be losing this category of customers. At the same time, we would be saving a lot in songs copyright expenses.

Without any kind of additional information, it's hard to determine what is the best strategy to apply from a business point of view. To avoid making any decision we could end up regretting, it's probably best to collect some precise data about the cost of songs rights; then we could run some numbers and have a better idea of the economical impact of each strategy.

## Finding the Artists Who Appear in the Greatest Number of Playlists

Let's find out what are the artists who appear in the greatest number of playlists.

In [12]:
%%sql

-- Create a temporary table with all the tracks and corresponding artists containted in the store's playlists 

WITH playlist_info AS(
    SELECT pt.playlist_id playlist_id,
           t.name track,
           ar.name artist
      FROM playlist_track pt
           INNER JOIN track t ON pt.track_id = t.track_id
           INNER JOIN album al ON al.album_id = t.album_id
           INNER JOIN artist ar ON ar.artist_id = al.artist_id
                            )


SELECT artist,
       COUNT(playlist_id) number_of_playlists
FROM (SELECT DISTINCT(artist), /*Collapse the temporary table selecting the unique artitsts.
                                 The result is a table with all unique artists together with all the playlists
                                 they appear on...*/ 

             playlist_id
        FROM playlist_info)
GROUP BY 1 -- ... now group by the artists and count the number of playlists they each of them appears on.
ORDER BY 2 DESC
LIMIT 10;

 * sqlite:///chinook.db
Done.


artist,number_of_playlists
Eugene Ormandy,7
The King's Singers,6
English Concert & Trevor Pinnock,6
Berliner Philharmoniker & Herbert Von Karajan,6
Academy of St. Martin in the Fields & Sir Neville Marriner,6
Yo-Yo Ma,5
Wilhelm Kempff,5
Ton Koopman,5
"Sir Georg Solti, Sumi Jo & Wiener Philharmoniker",5
Sir Georg Solti & Wiener Philharmoniker,5


With a total of 7, *Eugene Ormandy* is the artist which appears in the greatest number of playlists. I have honestly never heard of this artist, but consulting his [Wikipedia page](https://en.wikipedia.org/wiki/Eugene_Ormandy) one finds out he was an Hungarian-born conductor and violinist who spent most of his career working with the Philadelphia Orchestra. 

Taking a more attentive look, It's actually interesting to notice how virtually all the artists with strong playlists presence are or are part of orchestras. This is even more curious if we consider that, when we printed the number of purchases by genre, classical music tracks have been purchased a meager 4 times. Perhaps, this kind of music is sold for cheap by record companies; then the Chinook management might want to take advantage of this price and decides stuffing the playlists available on the store with classical music is a good idea.

## Assessing the Significance of the Media Type

Looking back at the database schema, we spot a table called *media_type* which must inevitably contain information on the different digital type of the tracks. Let's see what these types are and what is the most common.   

In [13]:
%%sql

-- Get each media type count

SELECT mt.name media_type, COUNT(*) tracks_purchased 
FROM invoice_line il 
     INNER JOIN track t ON il.track_id=t.track_id
     INNER JOIN media_type mt ON t.media_type_id = mt.media_type_id
GROUP BY 1
ORDER BY 2 DESC;

 * sqlite:///chinook.db
Done.


media_type,tracks_purchased
MPEG audio file,4259
Protected AAC audio file,439
Purchased AAC audio file,35
AAC audio file,21
Protected MPEG-4 video file,3


Strictly speaking, there are 5 different types of media; but it is more correct to say that there are 2 media types ([MPEG](https://en.wikipedia.org/wiki/Moving_Picture_Experts_Group) and [AAC](https://en.wikipedia.org/wiki/Advanced_Audio_Coding)) and each of them has an extra attribute (*protected* or *not protected*). The overwhelming majority (nearly 90%) of the tracks purchased are of MPEG type and none of these audio files are *protected*. 

On the other hand, protected audio files are the most common among the AAC type. 

## Purchased vs. Non-Purchased Tracks

Let's conclude by checking what portion of the full songs catalog available on the Chinook store has been purchased at least once.

In [14]:
%%sql

-- Create a temporary table with the number of tracks purchased at least once

WITH purch_tracks_count AS(
SELECT COUNT(DISTINCT(t.track_id)) purchased_tracks 
  FROM track t 
       INNER JOIN invoice_line il ON t.track_id = il.track_id)

-- Calculate the percentage of tracks purchased at least once

SELECT purchased_tracks, total_tracks, ROUND(CAST(purchased_tracks AS FLOAT) / total_tracks, 2) * 100
                                       AS percentage
  FROM purch_tracks_count t JOIN (SELECT COUNT(*) total_tracks
                  FROM track);

 * sqlite:///chinook.db
Done.


purchased_tracks,total_tracks,percentage
1806,3503,52.0


Only 52% of all the tracks available in the store have been purchased at least once. That looks like a quite low number. In retrospective, considering this figure, it might be a good idea to reduce the number of songs to put on the store. 